In [4]:
import sys
import os
from titanic_pipeline import config

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [38]:
import requests
import json
import pandas as pd
url = "http://127.0.0.1:8000/prediction"

headers = {
  'Content-Type': 'application/json'
}
raw_prod = pd.read_csv(config.DATASETS_DIR + 'prod_data.csv')

# for record in json.loads(raw_prod.to_json(orient='records')):
#     response = requests.request("POST", url, headers=headers, data=json.dumps(record))    

In [32]:

inputs = []
preds = []
with open(f"{config.BASE_DIR}/prediction_api.log") as f:
    for line in f:
        if "Prediction:" in line:
            payload = json.loads(line.split('Prediction:')[1])
            inputs.append(payload['input_raw'])
            preds.append(payload['prediction'])
            
prod = pd.DataFrame(inputs)
prod['prediction'] = preds

In [33]:
prod

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,prediction
0,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.3375,B5,S,2,?,1
1,3,"Rintamaki, Mr. Matti",male,35,0,0,STON/O 2. 3101273,7.125,?,S,?,?,0
2,3,"Nakid, Mr. Sahid",male,20,1,1,2653,15.7417,?,C,C,?,0
3,3,"Lyntakoff, Mr. Stanko",male,?,0,0,349235,7.8958,?,S,?,?,0
4,3,"Ford, Mr. Arthur",male,?,0,0,A/5 1478,8.05,?,S,?,?,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,3,"Buckley, Mr. Daniel",male,21,0,0,330920,7.8208,?,Q,13,?,0
652,3,"Yousif, Mr. Wazli",male,?,0,0,2647,7.225,?,C,?,?,0
653,1,"Andrews, Mr. Thomas Jr",male,39,0,0,112050,0.0,A36,S,?,?,0
654,3,"Flynn, Mr. John",male,?,0,0,368323,6.95,?,Q,?,?,0


## Model has been in production for one month
We collected the production data, but we do not have the ground truth available yet

In [9]:
import evidently

In [11]:
from evidently.dashboard import Dashboard
from evidently.dashboard.tabs import (
    DataDriftTab,
    DataQualityTab
)
import joblib
from evidently import ColumnMapping
column_mapping = ColumnMapping(target='survived', prediction='prediction')
model = joblib.load(config.PRODUCTION_MODEL_FILE)

raw_train  = pd.read_csv(config.DATASET_FILE)
target_train = raw_train[[config.TARGET]]
raw_train.drop(columns=config.TARGET, inplace=True)


In [18]:
dashboard = Dashboard(tabs=[DataQualityTab(), DataDriftTab()])
dashboard.calculate(reference_data=raw_train, current_data=raw_prod, column_mapping=column_mapping)
dashboard.save('../reports/data_quality_drift.html')

/Users/andre.bajorat/tec/titanic_pipeline/.titanic/lib/python3.9/site-packages/scipy/stats/_stats_py.py:6766: RuntimeWarning:

divide by zero encountered in true_divide



In [49]:
from evidently.dashboard.tabs import CatTargetDriftTab
model.get_params()
processor = model['preprocessor']
processed_prod = processor.transform(raw_prod.copy())
processed_prod = pd.DataFrame(processed_prod, columns=processor.get_feature_names_out())

processed_train = processor.transform(raw_train.copy())
processed_train = pd.DataFrame(processed_train, columns=processor.get_feature_names_out())

processed_prod['prediction'] = model.predict(raw_prod.copy())
processed_train['prediction'] = model.predict(raw_train.copy())

dashboard = Dashboard(tabs=[DataQualityTab(), DataDriftTab(), CatTargetDriftTab()])
dashboard.calculate(reference_data=processed_train, current_data=processed_prod, column_mapping=column_mapping)
dashboard.save('../reports/data_quality_drift_processed.html')

## Release of ground truth data for that month

In [53]:
target_prod = pd.read_csv(config.DATASETS_DIR + 'prod_truth.csv')

processed_prod['survived'] = target_prod[config.TARGET]
processed_train['survived'] = target_train[config.TARGET]

dashboard = Dashboard(tabs=[DataQualityTab(), DataDriftTab(), CatTargetDriftTab()])
dashboard.calculate(reference_data=processed_train, current_data=processed_prod, column_mapping=column_mapping)
dashboard.save('../reports/data_quality_drift_processed.html')

In [52]:
processed_prod

,pclass,age,sibsp,parch,fare,sex_male,cabin_C,cabin_D,cabin_E,cabin_Rare,...,title_Mr,title_Mrs,title_Other,pclass_nan,age_nan,sibsp_nan,parch_nan,fare_nan,prediction,survived
0,1.0,0.436325,0.000,0.000000,0.027091,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,1.0,0.248434,0.125,0.111111,0.059854,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
2,1.0,0.348643,0.000,0.000000,0.030022,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0
3,1.0,0.348643,0.000,0.000000,0.030608,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0
4,1.0,0.348643,0.000,0.000000,0.029468,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,1.0,0.260960,0.000,0.000000,0.029737,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
651,1.0,0.348643,0.000,0.000000,0.027471,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0
652,0.0,0.486430,0.000,0.000000,0.000000,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
653,1.0,0.348643,0.000,0.000000,0.026426,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0


In [55]:
rf_model = joblib.load(config.BASE_DIR + '/models/RandomForest_Best.sav')

In [56]:
rf_processed = processed_prod.copy()
rf_processed['prediction'] = rf_model.predict(raw_prod)

In [57]:
from evidently.dashboard.tabs import ClassificationPerformanceTab

dashboard = Dashboard(tabs=[CatTargetDriftTab(), ClassificationPerformanceTab()])
dashboard.calculate(reference_data=processed_prod, current_data=rf_processed, column_mapping=column_mapping)
dashboard.save('../reports/performance_rf_logreg.html')